In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import datetime
from sklearn import decomposition

In [2]:
data = pd.read_csv('./data/df_train.csv', encoding='utf-8')
data.head()

,instance_id,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description,is_trade,item_category_list_A,item_category_list_B,item_category_list_C,item_category_list_D,...,shop_star_level_5012,shop_star_level_5013,shop_star_level_5014,shop_star_level_5015,shop_star_level_5016,shop_star_level_5017,shop_star_level_5018,shop_star_level_5019,shop_star_level_5020,shop_star_level_nan
0,108641074714126964,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,5754713551599725161,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,842679481291040981,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,937088850059189027,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,7975697065017708072,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data.shape

(496509, 214)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496509 entries, 0 to 496508
Columns: 214 entries, instance_id to shop_star_level_nan
dtypes: float64(5), int64(209)
memory usage: 810.6 MB


In [5]:
train = data[~pd.isnull(data['is_trade'])]
test = data[pd.isnull(data['is_trade'])]
train.head()

,instance_id,shop_review_positive_rate,shop_score_service,shop_score_delivery,shop_score_description,is_trade,item_category_list_A,item_category_list_B,item_category_list_C,item_category_list_D,...,shop_star_level_5012,shop_star_level_5013,shop_star_level_5014,shop_star_level_5015,shop_star_level_5016,shop_star_level_5017,shop_star_level_5018,shop_star_level_5019,shop_star_level_5020,shop_star_level_nan
0,108641074714126964,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,5754713551599725161,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,842679481291040981,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,937088850059189027,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,7975697065017708072,1.0,1.0,1.0,1.0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
print(len(train))
print(len(test))

478138
18371


In [7]:
y = train.is_trade
X = train.drop(['instance_id', 'is_trade'], axis=1, inplace=False)

y_test = test.is_trade
X_test = test.drop(['instance_id', 'is_trade'], axis=1, inplace=False)

In [8]:
X.columns == X_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [9]:
X.columns 

Index(['shop_review_positive_rate', 'shop_score_service',
       'shop_score_delivery', 'shop_score_description', 'item_category_list_A',
       'item_category_list_B', 'item_category_list_C', 'item_category_list_D',
       'item_category_list_E', 'item_category_list_F',
       ...
       'shop_star_level_5012', 'shop_star_level_5013', 'shop_star_level_5014',
       'shop_star_level_5015', 'shop_star_level_5016', 'shop_star_level_5017',
       'shop_star_level_5018', 'shop_star_level_5019', 'shop_star_level_5020',
       'shop_star_level_nan'],
      dtype='object', length=212)

In [10]:
pca = decomposition.PCA(n_components=50)
pca.fit(np.array(X))
X = pca.transform(X)

#pca.fit(np.array(X_test))
X_test = pca.transform(X_test)

In [11]:
y.value_counts()

0.0    469117
1.0      9021
Name: is_trade, dtype: int64

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

In [13]:
clf = xgb.XGBClassifier(n_estimators=500, max_depth=3, learning_rate=0.1, subsample=0.8, silent=True, seed=1,
                           objective='binary:logistic', reg_alpha=0.5, reg_lambda=1, gamma=0, missing=None, colsample_bytree=0.3,
                       scale_pos_weight=1, min_child_weight=1, max_delta_step=0, base_socre=0.5)

In [14]:
clf.fit(X_train, y_train, eval_metric='auc', verbose=True, eval_set=[(X_val, y_val)], early_stopping_rounds=100)

[0]	validation_0-auc:0.514914
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.571656
[2]	validation_0-auc:0.57548
[3]	validation_0-auc:0.577533
[4]	validation_0-auc:0.576624
[5]	validation_0-auc:0.581599
[6]	validation_0-auc:0.587331
[7]	validation_0-auc:0.596791
[8]	validation_0-auc:0.600245
[9]	validation_0-auc:0.605344
[10]	validation_0-auc:0.604254
[11]	validation_0-auc:0.60526
[12]	validation_0-auc:0.608341
[13]	validation_0-auc:0.60922
[14]	validation_0-auc:0.610437
[15]	validation_0-auc:0.607816
[16]	validation_0-auc:0.609567
[17]	validation_0-auc:0.607465
[18]	validation_0-auc:0.608283
[19]	validation_0-auc:0.60906
[20]	validation_0-auc:0.609191
[21]	validation_0-auc:0.610963
[22]	validation_0-auc:0.611643
[23]	validation_0-auc:0.612448
[24]	validation_0-auc:0.613924
[25]	validation_0-auc:0.614052
[26]	validation_0-auc:0.614005
[27]	validation_0-auc:0.61389
[28]	validation_0-auc:0.612988
[29]	validation_0-auc:0.613485
[30]	validation_0-au

[259]	validation_0-auc:0.643122
[260]	validation_0-auc:0.643077
[261]	validation_0-auc:0.643117
[262]	validation_0-auc:0.642931
[263]	validation_0-auc:0.642961
[264]	validation_0-auc:0.642967
[265]	validation_0-auc:0.642924
[266]	validation_0-auc:0.642844
[267]	validation_0-auc:0.642984
[268]	validation_0-auc:0.64304
[269]	validation_0-auc:0.643085
[270]	validation_0-auc:0.643002
[271]	validation_0-auc:0.642998
[272]	validation_0-auc:0.64303
[273]	validation_0-auc:0.64309
[274]	validation_0-auc:0.64307
[275]	validation_0-auc:0.643192
[276]	validation_0-auc:0.643222
[277]	validation_0-auc:0.643296
[278]	validation_0-auc:0.643329
[279]	validation_0-auc:0.643386
[280]	validation_0-auc:0.643394
[281]	validation_0-auc:0.643515
[282]	validation_0-auc:0.643463
[283]	validation_0-auc:0.643446
[284]	validation_0-auc:0.643343
[285]	validation_0-auc:0.643336
[286]	validation_0-auc:0.643291
[287]	validation_0-auc:0.643273
[288]	validation_0-auc:0.643205
[289]	validation_0-auc:0.643101
[290]	valida

XGBClassifier(base_score=0.5, base_socre=0.5, booster='gbtree',
       colsample_bylevel=1, colsample_bytree=0.3, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.5, reg_lambda=1, scale_pos_weight=1, seed=1,
       silent=True, subsample=0.8)

In [15]:
clf.score(X_val, y_val)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9813025473710628

In [16]:
y_predict_val = clf.predict_proba(X_val)[:,1]

In [17]:
logloss = -sum(y_val * np.log(y_predict_val) + (1 - y_val) * np.log(1 - y_predict_val)) / len(y_val)
print(logloss)

0.09030599428740602


In [18]:
y_predict = clf.predict_proba(X_test)[:, 1]

In [19]:
y_predict

array([0.00487164, 0.01964608, 0.0209136 , ..., 0.01340945, 0.01887154,
       0.01412775], dtype=float32)

In [20]:
for i in y_predict:
    if i > 0.5:
        print(i)

In [21]:
test['predicted_score'] = y_predict

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
result = test[['instance_id', 'predicted_score']]
result.head()

,instance_id,predicted_score
478138,2475218615076601065,0.004872
478139,398316874173557226,0.019646
478140,6586402638209028583,0.020914
478141,1040996105851528465,0.023590
478142,6316278569655873454,0.010632


In [23]:
result['instance_id'] = result['instance_id'].astype('str')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
now = datetime.datetime.now()
path = './result_' + str(now.month) + str(now.day) + '.csv'
print(path)

./result_321.csv


In [25]:
result.to_csv(path, encoding='utf-8', index=False, sep=' ')